In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model

In [2]:
data = pd.read_excel("Fobal.xlsx")
num_jugadores=data.Jugador.dropna().nunique()-2
data_jugadores=data[:num_jugadores]
data_resultados=data[data.Jugador.isin(["Equipo 0","Equipo 1"])]

In [3]:
data_jugadores

,Jugador,2023-01-19 00:00:00,2023-01-26 00:00:00,2023-02-09 00:00:00
0,PabloB,0.0,1.0,0.0
1,Santi,0.0,0.0,NaN
2,Nuss,0.0,0.0,0.0
3,Ale,0.0,1.0,NaN
4,Delpe,0.0,1.0,0.0
5,German,1.0,NaN,NaN
6,Negri,1.0,0.0,1.0
7,Chehebar,1.0,0.0,0.0
8,Ivo,1.0,1.0,1.0
9,Charly,1.0,NaN,NaN


In [4]:
data_resultados

,Jugador,2023-01-19 00:00:00,2023-01-26 00:00:00,2023-02-09 00:00:00
19,Equipo 0,7.0,10.0,9.0
20,Equipo 1,12.0,10.0,6.0


In [5]:
X=[]
y=[]
for date in data.columns[1:]:
    eq0=[1 if i==0 else 0 for i in data_jugadores[date]]
    eq1=[1 if i==1 else 0 for i in data_jugadores[date]]
    X.append(eq0+[-i for i in eq1])
    X.append(eq1+[-i for i in eq0])
    y.append(data_resultados[data_resultados.Jugador=="Equipo 0"][date].iloc[0])
    y.append(data_resultados[data_resultados.Jugador=="Equipo 1"][date].iloc[0])
    

In [6]:
reg = linear_model.Ridge(alpha=1)
reg.fit(X, y)

Ridge(alpha=1)

In [7]:
def norm(l):
    mn=min(l)
    mx=max(l)
    return [round((i-mn)/(mx-mn)*10+90,1) for i in l]

In [8]:
ataq=reg.coef_[:num_jugadores]
ataq=norm(ataq)
defe=reg.coef_[num_jugadores:]
defe=norm(defe)
data_jugadores["Ataque"]=ataq
data_jugadores["Defensa"]=defe

C:\Users\pablo\AppData\Roaming\Python\Python38\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [9]:
data_jugadores[["Jugador","Ataque","Defensa"]]

,Jugador,Ataque,Defensa
0,PabloB,94.8,95.1
1,Santi,94.7,90.0
2,Nuss,94.1,94.6
3,Ale,95.4,90.6
4,Delpe,94.8,95.1
5,German,98.3,97.9
6,Negri,95.5,95.7
7,Chehebar,100.0,99.8
8,Ivo,96.1,96.3
9,Charly,98.3,97.9
